In [ ]:
import tensorflow as tf
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.keras.models import load_model


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
gpus

In [ ]:
encoder_input = tf.keras.layers.Input(shape=(256, 256, 3), name="encoder_input")

In [ ]:
encoder_conv_layer1 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), padding="same", strides=1, name="encoder_conv_1")(x)
# encoder_norm_layer1 = tf.keras.layers.BatchNormalization(name="encoder_norm_1")(encoder_conv_layer1)
encoder_activ_layer1 = tf.keras.layers.LeakyReLU(name="encoder_leakyrelu_1")(encoder_conv_layer1)

encoder_conv_layer2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", strides=2, name="encoder_conv_2")(encoder_activ_layer1)
# encoder_norm_layer2 = tf.keras.layers.BatchNormalization(name="encoder_norm_2")(encoder_conv_layer2)
encoder_activ_layer2 = tf.keras.layers.LeakyReLU(name="encoder_activ_layer_2")(encoder_conv_layer2)

encoder_conv_layer3 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", strides=2, name="encoder_conv_3")(encoder_activ_layer2)
# encoder_norm_layer3 = tf.keras.layers.BatchNormalization(name="encoder_norm_3")(encoder_conv_layer3)
encoder_activ_layer3 = tf.keras.layers.LeakyReLU(name="encoder_activ_layer_3")(encoder_conv_layer3)

encoder_conv_layer4 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding="same", strides=2, name="encoder_conv_4")(encoder_activ_layer3)
# encoder_norm_layer4 = tf.keras.layers.BatchNormalization(name="encoder_norm_4")(encoder_conv_layer4)
encoder_activ_layer4 = tf.keras.layers.LeakyReLU(name="encoder_activ_layer_4")(encoder_conv_layer4)

encoder_conv_layer5 = tf.keras.layers.Conv2D(filters=3, kernel_size=(3,3), padding="same", strides=1, name="encoder_conv_5")(encoder_activ_layer4)
# encoder_norm_layer5 = tf.keras.layers.BatchNormalization(name="encoder_norm_5")(encoder_conv_layer5)
encoder_activ_layer5 = tf.keras.layers.LeakyReLU(name="encoder_activ_layer_5")(encoder_conv_layer5)

In [ ]:
shape_before_flatten = tf.keras.backend.int_shape(encoder_activ_layer5)[1:]
encoder_flatten = tf.keras.layers.Flatten()(encoder_activ_layer5)

In [ ]:
shape_before_flatten

In [ ]:
latent_space_dim = 500
encoder_output = tf.keras.layers.Dense(units=latent_space_dim, name="encoder_output")(encoder_flatten)


In [ ]:
encoder = tf.keras.models.Model(x,encoder_output, name="encoder_model")

In [ ]:
encoder.summary()

In [ ]:
camera_vec_dim=100
camera_input  = tf.keras.layers.Input(shape=(camera_vec_dim), name="camera_input")
camera_dense_layer1 = tf.keras.layers.Dense(units=500, name="camera_dense_1",use_bias=False,activation=tf.keras.layers.LeakyReLU())(camera_input)
camera_dense_layer2 = tf.keras.layers.Dense(units=500, name="camera_dense_2",use_bias=False,activation=tf.keras.layers.LeakyReLU())(camera_dense_layer1)
# camera_dense_layer3 = tf.keras.layers.Dense(units=500, name="camera_dense_3",use_bias=False,kernel_initializer=initializer)(camera_dense_layer2)
camera_dense_layer4 = tf.keras.layers.Dense(units=500, name="camera_dense_4",use_bias=False,activation=tf.keras.layers.LeakyReLU())(camera_dense_layer2)
camera_dense_layer5 = tf.keras.layers.Dense(units=500, name="camera_dense_5",use_bias=False,activation=tf.keras.layers.LeakyReLU())(camera_dense_layer4)
# camera_dense_layer6 = tf.keras.layers.Dense(units=500, name="camera_dense_6")(camera_dense_layer5)
camera_output = tf.keras.layers.Dense(units=(latent_space_dim), name="camera_output")(camera_dense_layer5)

In [ ]:
camera_model = tf.keras.models.Model(camera_input,camera_output, name="camera_model")

In [ ]:
camera_model.summary()

In [ ]:
camera_model.trainable=False

In [ ]:
def concat(image_camera_enc):
    img,cam = image_camera_enc
    dense_input = img+cam
    return dense_input

In [ ]:
decoder_input1 = tf.keras.layers.Input(shape=(latent_space_dim), name="decoder_input1")
decoder_input2 = tf.keras.layers.Input(shape=(latent_space_dim), name="decoder_input2")
concat_layer = tf.keras.layers.Lambda(concat,name = "concat_layer")([decoder_input1, decoder_input2])
decoder_dense_layer1 = tf.keras.layers.Dense(units=np.prod(shape_before_flatten), name="decoder_dense_1")(concat_layer)
# decoder_dense_layer2 = tf.keras.layers.Dense(units=500, name="decoder_dense_2",activation="relu")(decoder_dense_layer1)
# decoder_dense_layer3 = tf.keras.layers.Dense(units=np.prod(shape_before_flatten), name="decoder_dense_3",activation="relu")(decoder_dense_layer2)

In [ ]:
decoder_reshape = tf.keras.layers.Reshape(target_shape=shape_before_flatten)(decoder_dense_layer1)

In [ ]:
decoder_conv_tran_layer1 = tf.keras.layers.Conv2DTranspose(filters=256, kernel_size=(3, 3), padding="same", strides=1, name="decoder_conv_tran_1")(decoder_reshape)
# decoder_norm_layer1 = tf.keras.layers.BatchNormalization(name="decoder_norm_1")(decoder_conv_tran_layer1)
decoder_activ_layer1 = tf.keras.layers.LeakyReLU(name="decoder_leakyrelu_1")(decoder_conv_tran_layer1)

decoder_conv_tran_layer2 = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=(3, 3), padding="same", strides=2, name="decoder_conv_tran_2")(decoder_activ_layer1)
# decoder_norm_layer2 = tf.keras.layers.BatchNormalization(name="decoder_norm_2")(decoder_conv_tran_layer2)
decoder_activ_layer2 = tf.keras.layers.LeakyReLU(name="decoder_leakyrelu_2")(decoder_conv_tran_layer2)

decoder_conv_tran_layer3 = tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=(3, 3), padding="same", strides=2, name="decoder_conv_tran_3")(decoder_activ_layer2)
# decoder_norm_layer3 = tf.keras.layers.BatchNormalization(name="decoder_norm_3")(decoder_conv_tran_layer3)
decoder_activ_layer3 = tf.keras.layers.LeakyReLU(name="decoder_leakyrelu_3")(decoder_conv_tran_layer3)

decoder_conv_tran_layer4 = tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=(3, 3), padding="same", strides=2, name="decoder_conv_tran_4")(decoder_activ_layer3)
# decoder_norm_layer4 = tf.keras.layers.BatchNormalization(name="decoder_norm_4")(decoder_conv_tran_layer4)
decoder_activ_layer4 = tf.keras.layers.LeakyReLU(name="decoder_leakyrelu_4")(decoder_conv_tran_layer4)

decoder_output = tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=(3, 3),activation="sigmoid", padding="same", strides=1, name="decoder_output")(decoder_activ_layer4)


In [ ]:
decoder = tf.keras.models.Model([decoder_input1,decoder_input2], decoder_output, name="decoder_model")

In [ ]:
decoder.summary()

In [ ]:
ae_input1 = tf.keras.layers.Input(shape=(256, 256, 3), name="AE_input1")
ae_input2 = tf.keras.layers.Input(shape=(camera_vec_dim), name="AE_input2")
ae_encoder_output = encoder(ae_input1)
ae_camera_output = camera_model(ae_input2)
ae_decoder_output = decoder([ae_encoder_output,ae_camera_output])

In [ ]:
ae = tf.keras.models.Model([ae_input1,ae_input2], ae_decoder_output, name="AE")

In [ ]:
ae.summary()

In [ ]:
import random
images=[]
for i in range(0,43):
    path = 'gtsd/Train/'+str(i)
    img = os.listdir(path)
    ch = random.sample(img,30)
    for j in ch:
        im = Image.open(path+"/"+j)
        im = im.resize((256,256))
        im = np.array(im)/255
        images.append(im)

img1 = Image.open("var_loss_img.jpg")
img1 = img1.resize((256,256))
img1 = np.array(img1)/255
images.append(img1)

img1 = Image.open("rt_sign1.jpg")
img1 = img1.resize((256,256))
img1 = np.array(img1)/255
images.append(img1)

In [ ]:
images = np.array(images)

In [ ]:
camera_input_shape=100
camera_params = np.zeros(shape=(camera_input_shape,))

In [ ]:
while camera_params.shape[0]<=1800:
    camera_params = np.vstack([camera_params,camera_params])

In [ ]:
camera_params = camera_params[:images.shape[0],:]

In [ ]:
images.shape

In [ ]:
camera_params.shape

In [ ]:
ae.compile(optimizer="adam",loss="mse",metrics="mape")

In [ ]:
for i in range(1,31):
    ae.fit([images,camera_params],images,epochs=5,batch_size=8)
    ae.save("auto_encoder.h5")

In [ ]:
plt.imshow(ae([images[200].reshape(1,256,256,3),tf.reshape(camera_params[0],[1,100])])[0])

In [ ]:
plt.imshow(images[200])

In [ ]:
import re
num = []
string =  os.listdir("sift_frames2")
for i in string:
    numbers = re.findall(r'\d+\.\d+|\d+', i)
    num.append(int(numbers[0]))
image_num = sorted(num)

In [ ]:
captured_images=[]
for i in range(0,len(image_num),10):
    img = Image.open("sift_frames2/"+str(image_num[i])+".jpg")
    img = img.resize((256,256))
    img = np.array(img)/255
    captured_images.append(img)
captured_images = np.array(captured_images)
input_img = Image.open("var_loss_img.jpg")
input_img = input_img.resize((256,256))
input_img = np.array(input_img)/255

In [ ]:
captured_images.shape

In [ ]:
camera_input = tf.keras.layers.Input(shape=(256, 256, 3), name="camer_input")
encoder_conv_layer1 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), padding="same", strides=1, name="encoder_conv_1")(camera_input)
# encoder_norm_layer1 = tf.keras.layers.BatchNormalization(name="encoder_norm_1")(encoder_conv_layer1)
encoder_activ_layer1 = tf.keras.layers.LeakyReLU(name="encoder_leakyrelu_1")(encoder_conv_layer1)

encoder_conv_layer2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", strides=2, name="encoder_conv_2")(encoder_activ_layer1)
# encoder_norm_layer2 = tf.keras.layers.BatchNormalization(name="encoder_norm_2")(encoder_conv_layer2)
encoder_activ_layer2 = tf.keras.layers.LeakyReLU(name="encoder_activ_layer_2")(encoder_conv_layer2)

encoder_conv_layer3 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", strides=2, name="encoder_conv_3")(encoder_activ_layer2)
# encoder_norm_layer3 = tf.keras.layers.BatchNormalization(name="encoder_norm_3")(encoder_conv_layer3)
encoder_activ_layer3 = tf.keras.layers.LeakyReLU(name="encoder_activ_layer_3")(encoder_conv_layer3)

encoder_conv_layer4 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding="same", strides=2, name="encoder_conv_4")(encoder_activ_layer3)
# encoder_norm_layer4 = tf.keras.layers.BatchNormalization(name="encoder_norm_4")(encoder_conv_layer4)
encoder_activ_layer4 = tf.keras.layers.LeakyReLU(name="encoder_activ_layer_4")(encoder_conv_layer4)

encoder_conv_layer5 = tf.keras.layers.Conv2D(filters=3, kernel_size=(3,3), padding="same", strides=1, name="encoder_conv_5")(encoder_activ_layer4)
# encoder_norm_layer5 = tf.keras.layers.BatchNormalization(name="encoder_norm_5")(encoder_conv_layer5)
encoder_activ_layer5 = tf.keras.layers.LeakyReLU(name="encoder_activ_layer_5")(encoder_conv_layer5)
encoder_flatten = tf.keras.layers.Flatten()(encoder_activ_layer5)
camera_output = tf.keras.layers.Dense(units=(camera_vec_dim), name="camera_output")(encoder_flatten)

In [ ]:
gen_cam = tf.keras.models.Model(camera_input,camera_output, name="gen_cam")

In [ ]:
gen_cam.summary()

In [ ]:
input_img = input_img.reshape(1,256,256,3)

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.001)

In [ ]:
camera_vectors=[]
cam_loss=[]
for epochs in range(1000):
    out_img = captured_images[i]
    X = captured_images[i].reshape(1,256,256,3)
    for i in range(177): 
        out_img = captured_images[i]
        X = captured_images[i].reshape(1,256,256,3)
        with tf.GradientTape() as tape:
            encoder.trainable=False
            decoder.trainable=False
            camera_model.trainable=False
            gen_cam.trainable=True 
            camera_vector = gen_cam(X)
            ae_output = ae([input_img,camera_vector])[0]
            loss = tf.math.reduce_mean(tf.math.square(out_img-ae_output))
        gradients = tape.gradient(loss, gen_cam.trainable_variables)
        optimizer.apply_gradients(zip(gradients, gen_cam.trainable_variables))
        print(i,epochs,loss) 
        with tf.GradientTape() as tape1:
            encoder.trainable=False
            decoder.trainable=False
            camera_model.trainable=True
            gen_cam.trainable=False
            camera_vector = gen_cam(X)
            ae_output = ae([input_img,camera_vector])[0]
            loss1 = tf.math.reduce_mean(tf.math.square(out_img-ae_output))
        gradients = tape1.gradient(loss1, camera_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, camera_model.trainable_variables))
        print(i,epochs,loss1) 
    camera_vectors.append(camera_vector.numpy()[0])

In [ ]:
np.save("camera_vectors.npy",camera_vectors)